In [1]:
from resnet import *
from baseline_cnn import *
from resnet import resnet50
import torch.nn as nn
import torch.optim as optim
import time
import pathlib
import torch
from evaluation import Evaluation
%load_ext autoreload
%autoreload 2
# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

class channelCopy(object):    
    def __call__(self, img):
        return torch.cat([img, img, img], 0)

#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize(512),transforms.ToTensor(),channelCopy()])


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
# train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
#                                                              p_val=p_val, p_test=p_test,
#                                                              shuffle=True, show_sample=False, 
#                                                              extras=extras)
train_loader, val_loader, test_loader = create_balanced_split_loaders(batch_size, seed, transform=transform,
                                                                         p_val=p_val, p_test=p_test,
                                                                         shuffle=True, show_sample=False,
                                                                         extras=extras, z_score=conf['z_score'])
# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = resnet50(pretrained=False, num_classes=14)
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
#criterion = nn.MultiLabelSoftMarginLoss() #TODO - loss criteria are defined in the torch.nn package
criterion = nn.MultiLabelSoftMarginLoss()
#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(model.parameters(),lr=0.00001) #TODO - optimizers are defined in the torch.optim package

CUDA is supported
Model on CUDA? True


In [2]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [3]:
# Set up folder for model saving
model_path = '{}/models/resnet50/{}/'.format(os.getcwd(), time.strftime("%Y%m%d-%H%M%S"))
model_pathlib = pathlib.Path(model_path)
if not model_pathlib.exists():
    pathlib.Path(model_pathlib).mkdir(parents=True, exist_ok=True)

#save loss in files for futher usage.
loss_path = '{}/losses/resnet50/{}/'.format(os.getcwd(), time.strftime("%Y%m%d-%H%M%S"))
loss_pathlib = pathlib.Path(loss_path)
if not loss_pathlib.exists():
    pathlib.Path(loss_pathlib).mkdir(parents=True, exist_ok=True)

In [ ]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []
loss_val_list = []
loss_val_min = float('inf')
N = 50
# Begin training procedure
for epoch in range(num_epochs):
    
    N_minibatch_loss = 0.0    

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):
#         if minibatch_count == 100:
#             break
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)

        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()

        # Perform the forward pass through the network and compute the loss
        outputs = model(images)
        loss = criterion(outputs, labels)
#         print('training',minibatch_count,loss)
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()
        
        # Update the weights
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
        
        #TODO: Implement validation
        if minibatch_count % N == 0:
            #switch to evaluate mode
            model.eval()
            with torch.no_grad():
                loss_val = 0
                for count_val, (images_val, labels_val) in enumerate(val_loader):
#                     if count_val ==10:
#                         break
                    images_val, labels_val = images_val.to(computing_device), labels_val.to(computing_device)
                    outputs_val = model(images_val)
                    loss_val += criterion(outputs_val, labels_val)
#                     print('val',count_val, (loss_val/(count_val+1)))
                loss_val /= count_val
                print('val',minibatch_count,loss_val)
                loss_val_list.append(loss_val.item())
                if loss_val < loss_val_min:
                    model_name = "epoch_{}-batch_{}-loss_{}-{}.pt".format(epoch, minibatch_count, loss_val, time.strftime("%Y%m%d-%H%M%S"))
                    torch.save(model.state_dict(), os.path.join(model_path, model_name))
                    loss_val_min = loss_val
                    
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss.item())
            N_minibatch_loss = 0.0
            
        prefix = 'resnet50_loss_'
        with open(os.path.join(loss_path, prefix+"training.txt"), "w") as f:
            for s in total_loss:
                f.write(str(s) +"\n")

        with open(os.path.join(loss_path, prefix+"training_ave.txt"), "w") as f:
            for s in avg_minibatch_loss:
                f.write(str(s) +"\n")

        with open(os.path.join(loss_path, prefix+"val.txt"), "w") as f:
            for s in loss_val_list:
                f.write(str(s) +"\n")
            
    print("Finished", epoch + 1, "epochs of training")
print("Training complete after", epoch, "epochs")

val 0 tensor(0.6917, device='cuda:0')
Epoch 1, average minibatch 0 loss: 0.013
val 50 tensor(0.1851, device='cuda:0')
Epoch 1, average minibatch 50 loss: 0.391
val 100 tensor(0.1819, device='cuda:0')
Epoch 1, average minibatch 100 loss: 0.189
val 150 tensor(0.1817, device='cuda:0')
Epoch 1, average minibatch 150 loss: 0.178
val 200 tensor(0.1833, device='cuda:0')
Epoch 1, average minibatch 200 loss: 0.183
val 250 tensor(0.1820, device='cuda:0')
Epoch 1, average minibatch 250 loss: 0.180
val 300 tensor(0.1821, device='cuda:0')
Epoch 1, average minibatch 300 loss: 0.179
val 350 tensor(0.1813, device='cuda:0')
Epoch 1, average minibatch 350 loss: 0.185
val 400 tensor(0.1846, device='cuda:0')
Epoch 1, average minibatch 400 loss: 0.187
val 450 tensor(0.1808, device='cuda:0')
Epoch 1, average minibatch 450 loss: 0.192
val 500 tensor(0.1810, device='cuda:0')
Epoch 1, average minibatch 500 loss: 0.196
val 550 tensor(0.1805, device='cuda:0')
Epoch 1, average minibatch 550 loss: 0.175
val 600 te

## inference

In [3]:
PATH = '/datasets/home/home-02/60/960/kshi/PA3/models/resnet50/20190214-113501/epoch_0-batch_1900-loss_0.17764276266098022-20190214-154931.pt'
model_test = resnet50(pretrained=False, num_classes=14)
model_test = model_test.to(computing_device)
model_test.load_state_dict(torch.load(PATH))

In [4]:
labels_all = []
predictions_all = []
model.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(computing_device), labels.to(computing_device)
        labels_all.append(labels)
        output = model_test(images)
        predictions = output > 0.5
        predictions_all.append(predictions)

labels = torch.cat(labels_all,0)
predctions = torch.cat(predictions_all,0)

In [5]:
eval = Evaluation(predctions.float(), labels)
print(eval.accuracy())
print(eval.accuracy().mean())
print()

TP [2088.0, 0.0, 2400.0, 0.0, 1045.0, 1170.0, 0.0, 990.0, 0.0, 0.0, 445.0, 305.0, 143.0, 0.0]
FP [18093.0, 0.0, 17781.0, 0.0, 19136.0, 19011.0, 0.0, 19191.0, 0.0, 0.0, 19736.0, 19876.0, 4828.0, 0.0]
TN [0.0, 19642.0, 0.0, 16574.0, 0.0, 0.0, 19954.0, 0.0, 19340.0, 19750.0, 0.0, 0.0, 14763.0, 20151.0]
FN [0.0, 539.0, 0.0, 3607.0, 0.0, 0.0, 227.0, 0.0, 841.0, 431.0, 0.0, 0.0, 447.0, 30.0]
tensor([0.1035, 0.9733, 0.1189, 0.8213, 0.0518, 0.0580, 0.9888, 0.0491, 0.9583,
        0.9786, 0.0221, 0.0151, 0.7386, 0.9985], device='cuda:0',
       dtype=torch.float64)
tensor(0.4911, device='cuda:0', dtype=torch.float64)


In [6]:
eval.precision()

tensor([0.1035,    nan, 0.1189,    nan, 0.0518, 0.0580,    nan, 0.0491,    nan,
           nan, 0.0221, 0.0151, 0.0288,    nan], device='cuda:0',
       dtype=torch.float64)

In [7]:
eval.recall()

tensor([1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 1.0000, 1.0000, 0.2424, 0.0000], device='cuda:0',
       dtype=torch.float64)